In [1]:
%jsroot on

In [2]:
#include<TFile.h>
#include<TH1.h>
#include<TLine.h>
#include<iostream>
#include<string>
#include<stdio.h>

In [3]:
TFile *f=new TFile("//Analysis/Output_HZZAnalysis/data.root");
//TFile *f = new TFile("//Analysis/Output_HZZAnalysis/ggH125_ZZ4lep.root");
mass_four_lep->Print(); // Test de lectura

TH1.Print Name  = mass_four_lep, Entries= 321, Total sum= 95


In [4]:
// gROOT->SetStyle("ATLAS"); // Establecer estilo.
gStyle->SetOptStat("emr"); // Dibujar estadisticas de grafico.
gStyle->SetOptFit(1112);

In [5]:
TCanvas *c1 = new TCanvas("c1","c1",10,10,900,600);

In [6]:
TH1F *m_four_lep = (TH1F*)f->Get("mass_four_lep");
m_four_lep->Draw();
// m_four_lep->Draw("E1"); // E1: Para representar los puntos con barras de error.

In [7]:
Int_t fit = 3; // Option 0 =

In [8]:
Double_t mybw(Double_t* x, Double_t* par)
{
    Double_t arg1 = 2/TMath::Pi();
    Double_t arg2 = pow(par[1],2)*pow(par[2],2); // Gamma=par[1] M=par[2]
    Double_t arg3 = pow((pow(x[0],2)-pow(par[2],2)),2);
    Double_t arg4 = pow(x[0],4)*(pow(par[1],2)/pow(par[2],2));
    return par[0]*arg1*arg2/(arg3+arg4);
}

In [9]:
if (fit == 0) {
    TF1 *f1 = new TF1("f1","mybw",115,140,3);
    f1->SetParameters(1,m_four_lep->GetMean(),m_four_lep->GetRMS());
    f1->SetParNames ("Constant","Mean","Sigma");
}
if (fit == 1) {TF1 *f1 = new TF1("f1","gaus",110,140);}
if (fit == 2) {TF1 *f1 = new TF1("f1","landau",110,140);}
if (fit == 3) {TF1 *f1 = new TF1("f1","gausn",110,140);}

In [10]:
m_four_lep->Fit("f1","R"); // Puede usarse V para agregar más detalles estadísticos.
f1->SetLineColor(kRed);

 FCN=4.58114 FROM MIGRAD    STATUS=CONVERGED      79 CALLS          80 TOTAL
                     EDM=6.57626e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.07067e+02   2.01209e+01   2.31676e-02   1.47042e-07
   2  Mean         1.24701e+02   1.30045e+00   1.19324e-03  -1.05361e-05
   3  Sigma        5.39714e+00   1.48300e+00   7.85902e-05   4.96810e-04


In [11]:
// Print the full information of the fit including covariance matrix.
f1->Print("V"); 

// Store the result in a ROOT file.
//f1->Write(); 

Formula based function:     f1 
                   f1 : gausn Ndim= 1, Npar= 3, Number= 100 
 Formula expression: 
	[Constant]*exp(-0.5*((x-[Mean])/[Sigma])*((x-[Mean])/[Sigma]))/(sqrt(2*pi)*[Sigma]) 
List of  Variables: 
Var   0                    x =    0.000000 
List of  Parameters: 
Par   0             Constant =  107.067367 
Par   1                 Mean =  124.700692 
Par   2                Sigma =    5.397143 
Expression passed to Cling:
	#pragma cling optimize(2)
Double_t TFormula____id5659692697328198837(Double_t *x,Double_t *p){ return p[0]*TMath::Exp(-0.5*((x[0]-p[1])/p[2])*((x[0]-p[1])/p[2]))/(TMath::Sqrt(2*3.141593)*p[2]) ; }


In [12]:
minx = f1->GetXmin(); // Mínimo en x del ajuste.
maxx = f1->GetXmax(); // Máximo en x del ajuste.

mode_x = f1->GetMaximumX(minx,maxx); // Coordenada x del máximo en el ajuste dentro del intervalo [minx,maxx].
mode_y = f1->Eval(mode_x); // Coordenada y del máximo.

upper_halfwidth = f1->GetX(mode_y/2.0,mode_x,maxx);
lower_halfwidth = f1->GetX(mode_y/2.0,minx,mode_x);

width = upper_halfwidth-lower_halfwidth;

cout    << "Coordenada del punto máximo en el ajuste: (" << mode_x << "," << mode_y << ")." << endl
        << "Intersecciones en m4l a média altura: " << lower_halfwidth << " GeV" << " y " << upper_halfwidth << " GeV." << endl
        << "Ancho de decaimiento, \u0393 = " << width << " GeV." << endl
        << "Tiempo de vida media, \u03C4 = " << 4.135667696E-15/(width*1E9) << " s." << endl;

TLine *l1 = new TLine(lower_halfwidth,mode_y/2.0,upper_halfwidth,mode_y/2.0);
l1->SetLineColor(kGreen);
l1->SetLineStyle(7);
l1->Draw("same");

TLine *l2 = new TLine(mode_x,mode_y,mode_x,0);
l2->SetLineColor(kGreen);
l2->SetLineStyle(7);
l2->Draw("same");

c1->Draw();

Coordenada del punto máximo en el ajuste: (124.701,7.91413).
Intersecciones en m4l a média altura: 118.346 GeV y 131.055 GeV.
Ancho de decaimiento, Γ = 12.7093 GeV.
Tiempo de vida media, τ = 3.25405e-25 s.


#### OJO Revisar los orden de magnitud y unidades.